In [2]:
import os
import json
import pandas as pd
import traceback
import re

def fix_json_content(content):
    """Fix common JSON formatting issues."""
    content = re.sub(r'([{,])\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*:', r'\1"\2":', content)
    content = re.sub(r'}\s*{', '},{', content)
    content = content.replace(',]', ']')
    content = content.replace(',}', '}')
    content = content.replace('\\"', '"')
    content = content.replace(',"https"://', ', https://').replace('\',"https"://', ', https://')
    content = content.replace('"https"://', 'https://')
    return content

def process_json_files(base_path):
    data = []
    if not os.path.exists(base_path) or not os.path.isdir(base_path):
        print(f"Directory {base_path} does not exist.")
        return data

    for root, _, files in os.walk(base_path):
        for file in files:
            if not file.endswith(".json") and "." in file:
                print(f"Skipping {file} because it is not a JSON file.")
                continue
                
            file_path = os.path.join(root, file)

            print(file_path)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    print('inside')
                    json_data = json.load(f)                    

                    #Activate below to combine multiple json files into one - Deals with errors in json files
                    # try:
                    #     # json_data = json.loads(fixed_content)
                    #     json_data = json.load(f)
                    # except json.JSONDecodeError as e:
                    #     print(f"First JSON parse attempt failed: {str(e)}")
                    #     # Show the problematic text around the error position
                    #     if hasattr(e, 'pos') and e.pos is not None:
                    #         start_pos = max(0, e.pos - 50)
                    #         end_pos = min(len(fixed_content), e.pos + 50)
                    #         problematic_text = fixed_content[start_pos:end_pos]
                    #         print(f"Problematic text around position {e.pos}: '{problematic_text}'")
                    #         if e.pos < len(fixed_content):
                    #             print(f"Character at error position: '{fixed_content[e.pos]}' (ASCII: {ord(fixed_content[e.pos])})")
                        
                    #     try:
                    #         # Remove control characters more aggressively
                    #         cleaned_content = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', fixed_content)
                    #         json_data = json.loads(cleaned_content)
                    #     except json.JSONDecodeError as e2:
                    #         print(f"Second JSON parse attempt failed: {str(e2)}")
                    #         # Show the problematic text around the error position for second attempt
                    #         if hasattr(e2, 'pos') and e2.pos is not None:
                    #             start_pos = max(0, e2.pos - 50)
                    #             end_pos = min(len(cleaned_content), e2.pos + 50)
                    #             problematic_text = cleaned_content[start_pos:end_pos]
                    #             print(f"Problematic text around position {e2.pos}: '{problematic_text}'")
                    #             if e2.pos < len(cleaned_content):
                    #                 print(f"Character at error position: '{cleaned_content[e2.pos]}' (ASCII: {ord(cleaned_content[e2.pos])})")
                            
                    #         try:
                    #             # Try to fix common JSON issues
                    #             corrected_string = re.sub(r'{([^{}]*)}', r'"\1"', cleaned_content)
                    #             json_data = json.loads(corrected_string)
                    #         except json.JSONDecodeError as e3:
                    #             print(f"All JSON parse attempts failed for {file_path}")
                    #             print(f"Final error: {str(e3)}")
                    #             # Show the problematic text around the error position for final attempt
                    #             if hasattr(e3, 'pos') and e3.pos is not None:
                    #                 start_pos = max(0, e3.pos - 50)
                    #                 end_pos = min(len(corrected_string), e3.pos + 50)
                    #                 problematic_text = corrected_string[start_pos:end_pos]
                    #                 print(f"Final problematic text around position {e3.pos}: '{problematic_text}'")
                    #                 if e3.pos < len(corrected_string):
                    #                     print(f"Final character at error position: '{corrected_string[e3.pos]}' (ASCII: {ord(corrected_string[e3.pos])})")
                    #             continue  # Skip this file and move to the next one
                
                # if not isinstance(json_data, list):
                #     json_data = [json_data]
                
                for entry in json_data:
                    if not isinstance(entry, dict):
                        continue
                    # if entry.get("answer", ""):
                    #     answer_text = entry.get("answer", "")
                    # else:
                    #     answer_text = entry.get("response", "")
                    data.append([
                        # root.split("peer-review")[1]+'/'+file,
                        entry.get("id", ""),
                        entry.get("question", ""),
                        # answer_text,
                        entry.get("answer", ""),
                        # f"{entry.get('topic/subtopic', '')}",
                        # entry.get("reference", ""),
                        entry.get("origin_file", "")
                    ])
                    
            except Exception as e:
                print(f"Error processing {file_path}: {str(e)}")
                traceback.print_exc()
                if "Expecting property name" in str(e) or "Expecting ',' delimiter" in str(e):
                    print("This appears to be a JSON formatting error. Please check the file for proper JSON syntax.")
    
    return data

def save_to_excel(data, output_path):
    if not data:
        print("No JSON data found to write to Excel.")
        return False
        
    try:
        df = pd.DataFrame(data, columns=["id", "question", "answer", "origin_file"])
        df.to_excel(output_path, index=False)
        print(f"Excel file created at: {output_path}")
        return True
    except Exception as e:
        print(f"Error saving Excel file: {str(e)}")
        traceback.print_exc()
        return False

if __name__ == "__main__":
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    trial01_path = os.path.join(desktop_path, "json")
    output_path = os.path.join(desktop_path, "DRACO_conference.xlsx")
    
    data = process_json_files(trial01_path)
    save_to_excel(data, output_path)

C:\Users\soyrl\Desktop\json\merged.json
inside
Excel file created at: C:\Users\soyrl\Desktop\DRACO_conference.xlsx
